## Zero or few shot learning to data labeling

In [1]:
!pip install openai

     |███████████████████████████████▎| 153 kB 6.5 MB/s eta 0:00:01     |████████████████████████████████| 157 kB 6.5 MB/s 
     |████████████████████████████████| 158 kB 33.7 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 26.4 MB/s eta 0:00:01     |███████████████████████████████▏| 235 kB 26.4 MB/s eta 0:00:01
  Created wheel for openai: filename=openai-0.10.4-py3-none-any.whl size=169207 sha256=f920a923f217c50aeec3f99670d0cefaf366c0b035293fa65970de7faa72c218
  Stored in directory: /Users/bot/Library/Caches/pip/wheels/8b/93/f8/5e75b386c7b042895640bfff87251b80c8986f8eca8f5a41d2
Successfully built openai
You should consider upgrading via the '/Users/bot/.pyenv/versions/3.7.4/bin/python3 -m pip install --upgrade pip' command.


In [63]:
!wget https://raw.githubusercontent.com/b2wdigital/b2w-reviews01/main/B2W-Reviews01.csv

--2021-09-27 21:14:34--  https://raw.githubusercontent.com/b2wdigital/b2w-reviews01/main/B2W-Reviews01.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49453175 (47M) [text/plain]
Saving to: ‘B2W-Reviews01.csv.1’

B2W-Reviews01.csv.1 100%[===================>]  47,16M  33,5MB/s    in 1,4s    

2021-09-27 21:14:38 (33,5 MB/s) - ‘B2W-Reviews01.csv.1’ saved [49453175/49453175]



In [64]:
!ls -lah

total 101752
drwxr-xr-x   6 bot  staff   192B Sep 27 21:14 .
drwxr-xr-x  57 bot  staff   1.8K Sep 20 21:46 ..
drwxr-xr-x   4 bot  staff   128B Sep 27 21:13 .ipynb_checkpoints
-rw-r--r--   1 bot  staff   2.2M Sep 27 21:13 B2W-Reviews01.csv
-rw-r--r--   1 bot  staff    47M Sep 27 21:14 B2W-Reviews01.csv.1
-rw-r--r--   1 bot  staff    17K Sep 27 21:13 zero or few shot learning.ipynb


In [13]:
import os
import openai
openai.organization = os.getenv("OPENAI_ORG")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Engine.list()

<OpenAIObject list at 0x10c67b590> JSON: {
  "data": [
    {
      "created": null,
      "id": "ada",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "ready_replicas": null,
      "replicas": null
    },
    {
      "created": null,
      "id": "babbage",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "ready_replicas": null,
      "replicas": null
    },
    {
      "created": null,
      "id": "curie",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "ready_replicas": null,
      "replicas": null
    },
    {
      "created": null,
      "id": "curie-instruct-beta",
      "max_replicas": null,
      "object": "engine",
      "owner": "openai",
      "permissions": null,
      "ready": true,
      "ready_replicas": null,
      "replic

In [50]:
prompt = """
1. gostei muito do jogo - positivo
2. esse prato tinha um gosto ruim - negativo
3. contemplei a natureza, muito bom! - 
"""

parameters = {
  "prompt": prompt,
  "max_tokens": 100,
  "temperature": 0.85,
  "top_p": 1,
  "n": 1,
  "stream": False,
  "logprobs": None,
  "frequency_penalty": 0,
  "stop": ""
}


response = openai.Completion.create(
  engine="ada",
  **parameters
)

response.choices[0].text

In [53]:
response.choices[0].text
# response

"4. 'gostei muito' - positivo\n5. olou muito por vinte minutos - negativo\n6. eu já fui 'somente ouvi' - negativo\n7. dou 'adversadores' - positivo\n8. acho que estou 'ser muito sinistro' - positivo\n9. passei 'completamente' - neg"

In [54]:
import pandas as pd

In [161]:
df = pd.read_csv("./B2W-Reviews01.csv")

df["sentiment"] = df["recommend_to_a_friend"].apply(lambda x: "positivo" if str(x).lower() == "yes" else "negativo")

In [166]:
cols = ["product_name","site_category_lv1", "review_title", "review_text", "recommend_to_a_friend", "sentiment"]

In [162]:
df[["sentiment","recommend_to_a_friend"]].value_counts()

sentiment  recommend_to_a_friend
positivo   Yes                      96368
negativo   No                       35987
dtype: int64

In [165]:
df[["sentiment","recommend_to_a_friend"]].sample(n=5)

,sentiment,recommend_to_a_friend
119771,negativo,No
114242,negativo,No
72692,positivo,Yes
97766,positivo,Yes
34955,positivo,Yes


In [248]:
random_state = 21
no_df = df[df["recommend_to_a_friend"] == "No"].sample(n=50, random_state=random_state)
yes_df = df[df["recommend_to_a_friend"] == "Yes"].sample(n=50, random_state=random_state)

label_df = pd.concat([no_df, yes_df])

label_df.shape

(100, 15)

In [206]:
[1]

'Vale muito pelo preço! Não conheço outros espremedores, talvez esse seja um pouco barulhento! Mas cumpre o que promete.'

In [245]:
def autolabel(text: str, sample_df: pd.DataFrame):

    prompt = generate_prompt(text, sample_df)


    #prompt="This is a tweet sentiment classifier\nTweet: \"I loved the new Batman movie!\"\nSentiment: Positive\n###\nTweet: \"I hate it when my phone battery dies\"\nSentiment: Negative\n###\nTweet: \"My day has been 👍\"\nSentiment: Positive\n###\nTweet: \"This is the link to the article\"\nSentiment: Neutral\n###\nTweet text\n\n\n1. \"I loved the new Batman movie!\"\n2. \"I hate it when my phone battery dies\"\n3. \"My day has been 👍\"\n4. \"This is the link to the article\"\n5. \"This new music video blew my mind\"\n\n\nTweet sentiment ratings:\n1: Positive\n2: Negative\n3: Positive\n4: Neutral\n5: Positive\n\n\n###\nTweet text\n\n\n1. \"I can't stand homework\"\n2. \"This sucks. I'm bored 😠\"\n3. \"I can't wait for Halloween!!!\"\n4. \"My cat is adorable ❤️❤️\"\n5. \"I hate chocolate\"\n\n\nTweet sentiment ratings:\n1.",


    # parameters = {
    #   "prompt": prompt,
    #   "max_tokens": 6,
    #   "temperature": 0,
    #   "top_p": 1.0,
    # #   "n": 1,
    # #   "stream": False,
    #   "logprobs": None,
    #   "frequency_penalty": 0,
    #   "presence_penalty": 0,
    #   "stop": ["###"]
    # }

    parameters = dict(
      engine="davinci",
      prompt=prompt,
      temperature=0.3,
      max_tokens=10,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=["\n"]
    )


    response = openai.Completion.create(
      **parameters
    )

    label_suggestion = response.choices[0].text
    return label_suggestion.strip().lower()

In [246]:
def generate_prompt(text, sample):

    text_examples = sample["review_text"].values
    label_examples = sample["sentiment"].values

    txt = []
    for _text, _label in list(zip(text_examples, label_examples)):
        txt.append(f"""Text: "{_text}"\nCategory: {_label}\n""")

    txt.append(f"""Text: "{text}"\nCategory:""")
    s = "\n".join(txt)
    return s

# prompt = f"""Text: "Entrega estava prevista para 7 dias e chegou em mais de 20 depois de muita reclamação"
# Category: Negativo

# Text: "O produto em si é muito ruim, realmente frustrado"
# Category: Negativo

# Text: "Ótimo custo benefício, entrega magnífica, super rápido"
# Category: Positivo

# Text: "Produto boa qualidade, faz jus ao preço. Gostei do produto"
# Category: Positivo

# Text: "{text}"
# Category:"""

generate_prompt("opa", label_df)

'Text: "Entrega estava prevista para 7 dias e chegou em mais de 20 depois de muita reclamação. As peças vieram com as pontas amassadas e algumas partes arranhadas, mas passei tanto stress com a entrega que deixei pra lá. A bancada é resistente, porém o parceiro não cumpre com o prometido."\nCategory: negativo\n\nText: "o produto era pra ter chegado a dois dias sendo que o prazo de entrega foi 30 dias , mas nao chegou ainda"\nCategory: negativo\n\nText: "Recebi o produto dentro do prazo estabelecido, porém, não vieram peças suficientes para montagem das cadeiras e a mesa estava com um arranhão. Não recomendo o produto, pois o mesmo parece frágil e veio com todos os problemas citados acima."\nCategory: negativo\n\nText: "Esse estojo Ele tem dois dividoria pois na foto só tá para ver um zíper"\nCategory: positivo\n\nText: "Vale muito pelo preço! Não conheço outros espremedores, talvez esse seja um pouco barulhento! Mas cumpre o que promete."\nCategory: positivo\n\nText: "Produto de qualid

In [239]:
s

'Text: "Nao gostei da qualidade do produto, veio faltando parafusos e peças a madeira de qualidade muito ruim e furacão errada."\nCategory: negativo\n\nText: "Produto bom em relação a custo beneficio vem 10 peças e as assadeiras tem um bom tamanho."\nCategory: positivo\n\nText: "Vale muito pelo preço! Não conheço outros espremedores, talvez esse seja um pouco barulhento! Mas cumpre o que promete."\nCategory: positivo\n\nText: "o produto era pra ter chegado a dois dias sendo que o prazo de entrega foi 30 dias , mas nao chegou ainda"\nCategory: negativo\n\nText: "o produto era pra ter chegado a dois dias sendo que o prazo de entrega foi 30 dias , mas nao chegou ainda"\nCategory:'

In [249]:
true = []
pred = []
idx = []

sample_df = label_df.sample(n=4, random_state=random_state)
for _idx, row in label_df[cols].iterrows():
    _pred = autolabel(row.review_text, sample_df)
    _true = row.sentiment
    #print(row.review_text, _pred, _true)
    pred.append(_pred)
    true.append(_true)
    idx.append(_idx)

In [194]:
idx[-1]

102039

In [201]:
label_df[label_df.index == 102039][["sentiment", "review_text"]].values

array([['positivo',
        'O produto em si é muito bom, realmente de qualidade mas como já reclamado por outros clientes a caixa do relógio veio de papelão, diferente das que os relógios costumam ser entregues (inclusive os próprios relógios da Casio). Comprar para dar como presente realmente deixa muito a desejar, não dá para utilizar a embalagem.']],
      dtype=object)

In [261]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, average_precision_score

In [250]:
accuracy_score(true, pred)

0.91

In [257]:
def convert_to_int(i): return 0 if i == "negativo" else 1
t = [convert_to_int(i) for i in true]
p = [convert_to_int(i) for i in pred]

In [258]:
roc_auc_score(t, p)

0.91

In [262]:
average_precision_score(t, p)

0.8698039215686275

In [259]:
print(
    classification_report(true, pred)
)

              precision    recall  f1-score   support

    negativo       0.92      0.90      0.91        50
      neutro       0.00      0.00      0.00         0
    positivo       0.92      0.92      0.92        50

    accuracy                           0.91       100
   macro avg       0.61      0.61      0.61       100
weighted avg       0.92      0.91      0.91       100



/Users/bot/.pyenv/versions/3.7.4/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bot/.pyenv/versions/3.7.4/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bot/.pyenv/versions/3.7.4/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [271]:
for p,t,i in list(zip(pred, true, idx)):
    if p != t:
        print(i, p, t)
        print(
            label_df[label_df.index == i][["sentiment", "review_text"]].values[0]
        )
        print()
    pass

119315 positivo negativo
['negativo'
 'Pelo preço que eu paguei, está servindo muito bem as minhas necessidades, excelente produto']

5326 positivo negativo
['negativo' nan]

106393 positivo negativo
['negativo' nan]

90327 neutro negativo
['negativo'
 'poderia ter mais cores a disposição na situação deste produto']

24187 positivo negativo
['negativo'
 'Recomendo o produto e também o site para a compra, o mesmo foi entregue antes do prazo previsto, muito bom.']

102039 negativo positivo
['positivo'
 'O produto em si é muito bom, realmente de qualidade mas como já reclamado por outros clientes a caixa do relógio veio de papelão, diferente das que os relógios costumam ser entregues (inclusive os próprios relógios da Casio). Comprar para dar como presente realmente deixa muito a desejar, não dá para utilizar a embalagem.']

14287 negativo positivo
['positivo'
 'Bem embalado, porém eu mesma fui tentar montar e as peças não vem todas numeradas dificultando muito!! Da trabalho montar, não é

['negativo'
 'Pelo preço que eu paguei, está servindo muito bem as minhas necessidades, excelente produto']
